In [9]:
import json, re, uuid, random
import os, tempfile, hashlib
from tqdm import tqdm
import faiss
import pandas as pd
import numpy as np
import torch
from pathlib import Path
import nltk
import string
from dataclasses import dataclass
from datasets import Dataset
from evaluate import load as hf_load
from typing import Dict, List, Any, Tuple
from nltk.tokenize import sent_tokenize
from rank_bm25 import BM25Okapi
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft import PeftModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSequenceClassification
from typing import Dict, List
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import evaluate
from nltk.corpus import stopwords


In [16]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def free_cuda():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [17]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))


CUDA available: True
GPU name: Tesla T4


In [10]:
SEED = 42  # any integer you like
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_grad_enabled(False)

torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [12]:
# Read in train / val/ test datasets
train_df = pd.read_csv("medquad_train.csv")
val_df = pd.read_csv("medquad_val.csv")
test_df = pd.read_csv("medquad_test.csv")

In [13]:
# Load any generator (tokenizer + model) by path

def load_generator(model_dir: str):
    """
    Loads a generator (tokenizer + CausalLM) from a local directory.
    Works for both BASELINE2_DIR (base BioGPT) and FULLMODEL_DIR (merged LoRA).
    """
    tok = AutoTokenizer.from_pretrained(model_dir)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    gen = AutoModelForCausalLM.from_pretrained(model_dir)
    gen.config.pad_token_id = tok.pad_token_id
    gen = gen.to(device).eval()
    return tok, gen

In [14]:
BASELINE2_DIR = "./biogpt_base"                    # zero-shot BioGPT (Baseline 2)
FULLMODEL_DIR  = "./finetuned_biogpt"    # merged LoRA (Full model)

tok_base, gen_base = load_generator(BASELINE2_DIR)
tok_full, gen_full = load_generator(FULLMODEL_DIR)


In [19]:
# ========= BM25 + PubMedBERT Retrieval ===========
# BM25 Baseline (lexical)

# Rebuild a richer 'doc' column (question + answer + metadata)
def build_doc_row(row):
    parts = []
    # Put question first so BM25 matches user phrasing better
    q = str(row.get("question", "")).strip()
    a = str(row.get("answer", "")).strip()
    if q: parts.append(f"Q: {q}")
    if a: parts.append(f"A: {a}")

    meta_bits = []
    for lab, key in [("Entity", "entity"), ("Type", "qtype"), ("Source", "source"), ("URL", "url")]:
        val = str(row.get(key, "") or "").strip()
        if val:
            meta_bits.append(f"{lab}: {val}")
    if meta_bits:
        parts.append("\n".join(meta_bits))

    return "\n".join(parts).strip()

train_df = train_df.copy()
train_df["doc"] = train_df.apply(build_doc_row, axis=1)

# === Example Result ===
# Q: What are the symptoms of pneumonia?
# A: The most common symptoms include cough, fever, chills, and difficulty breathing.
# Entity: Pneumonia
# Source: MedlinePlus
# URL: https://medlineplus.gov/pneumonia.html

# Build BM25 index over the new 'doc'
## strip puncturation and keep medically relevant alphanumeric tokens clean
_word_re = re.compile(r"[A-Za-z0-9]+(?:[-'][A-Za-z0-9]+)?")

def simple_tokenize(text: str):
    if not isinstance(text, str): text = "" if text is None else str(text)
    return _word_re.findall(text.lower())

bm25_corpus_texts  = train_df["doc"].tolist()
bm25_corpus_tokens = [simple_tokenize(t) for t in bm25_corpus_texts]
bm25 = BM25Okapi(bm25_corpus_tokens)
_train_rows = train_df.reset_index(drop=True)


# BM25 computes a relevance score for every document in the index, comparing those tokens 
# against each doc's tokens
def bm25_search(query: str, k: int = 20):
    q_tokens = simple_tokenize(query)
    scores = bm25.get_scores(q_tokens)
    if len(scores) == 0:
        return train_df.iloc[[]].assign(bm25_score=[])
    top_idx = np.argsort(scores)[::-1][:k]
    out = _train_rows.iloc[top_idx].copy()
    out["bm25_score"] = np.asarray(scores)[top_idx]
    return out


# --- Simple extractive baseline using the top hit's answer ---
def first_sentences(text: str, n: int = 3):
    sents = re.split(r'(?<=[.!?])\s+', (text or "").strip())
    return " ".join(sents[:n]).strip()

def baseline_bm25_extractive(user_query: str, k_retrieve: int = 5, n_sents: int = 3):
    hits = bm25_search(user_query, k=k_retrieve)
    if hits.empty:
        return {
            "answer": "Sorry, I couldn’t find an answer in my knowledge base.",
            "source": "",
            "url": "",
            "retrieval_candidates": []
        }
    top = hits.iloc[0]
    extract = first_sentences(top["answer"], n=n_sents) or top["answer"]
    return {
        "answer": extract,
        "source": str(top.get("source", "")),
        "url": str(top.get("url", "")),
        "retrieval_candidates": hits[["question","source","url","bm25_score"]].to_dict("records")
    }

# 1) Dense retriever (Sentence-Transformers + FAISS)

# --- 1.1 Load PubMedBERT bi-encoder (MS MARCO tuned) ---
DENSE_MODEL = "pritamdeka/S-PubMedBert-MS-MARCO"
dense = SentenceTransformer(DENSE_MODEL, device=device)

# --- 1.2 Build dense corpus on the same text used by BM25 ---
# Use the 'doc' (Q + A + meta) or just 'answer'. Start with 'doc' for recall.
corpus_texts = train_df["doc"].astype(str).tolist()

# --- 1.3 Encode corpus (normalized for cosine) ---
dense_emb = dense.encode(
    corpus_texts, batch_size=128, normalize_embeddings=True,
    show_progress_bar=True
).astype("float32")

# --- 1.4 FAISS index (inner product == cosine due to normalized vectors) ---
dim = dense_emb.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(dense_emb)

# Row mapping
_dense_rows = train_df.reset_index(drop=True)

def dense_search(query: str, k: int = 200) -> pd.DataFrame:
    qv = dense.encode([query], normalize_embeddings=True).astype("float32")
    scores, idx = index.search(qv, k)
    idx, scores = idx[0], scores[0]
    out = _dense_rows.iloc[idx].copy()
    out["dense_score"] = scores
    return out

# Hybrid retrieval (BM25 + Dense Fusion)
from sklearn.preprocessing import minmax_scale

def hybrid_search(query: str, k_bm25=200, k_dense=200, k_final=50, alpha=0.5) -> pd.DataFrame:
    """
    alpha = weight for dense (0..1). 0.5 = equal.
    """
    bm = bm25_search(query, k=k_bm25)[["question","answer","source","url","bm25_score"]].copy()
    de = dense_search(query, k=k_dense)[["question","answer","source","url","dense_score"]].copy()

    merged = bm.merge(de, how="outer", on=["question","answer","source","url"])
    merged["bm25_score"] = merged["bm25_score"].fillna(0.0)
    merged["dense_score"] = merged["dense_score"].fillna(0.0)

    merged["bm25_s"]  = minmax_scale(merged["bm25_score"].to_numpy(dtype=float), copy=True)
    merged["dense_s"] = minmax_scale(merged["dense_score"].to_numpy(dtype=float), copy=True)
    merged["hybrid"]  = (1 - alpha) * merged["bm25_s"] + alpha * merged["dense_s"]

    return merged.sort_values("hybrid", ascending=False).head(k_final).reset_index(drop=True)


# Cross-encoder re-ranker (PubMedBERT Cross-Encoder)
RERANK_NAME = "ncbi/MedCPT-Cross-Encoder"  # PubMedBERT cross-encoder
ce_tok = AutoTokenizer.from_pretrained(
    RERANK_NAME,
    model_max_length=512,
    truncation_side="right"
)
ce_model = AutoModelForSequenceClassification.from_pretrained(RERANK_NAME).to(device).eval()

def clip_passage(text: str, max_passage_tokens: int = 400) -> str:
    # encode *without* specials, clip, decode back
    ids = ce_tok.encode(text or "", add_special_tokens=False)[:max_passage_tokens]
    return ce_tok.decode(ids, skip_special_tokens=True)

@torch.inference_mode()
def rerank_cross_encoder(
    query: str,
    df_candidates: pd.DataFrame,
    top_n: int = 8,
    batch_size: int = 16,
    max_length: int = 512,
    max_passage_tokens: int = 400,     # leave room for query + specials
):
    if df_candidates.empty:
        return df_candidates

    # Prefer to preserve the *entire query* and truncate only the passage
    q = str(query or "")
    d_texts = [clip_passage(str(a), max_passage_tokens=max_passage_tokens)
               for a in df_candidates["answer"].astype(str).tolist()]

    scores = []
    for i in range(0, len(d_texts), batch_size):
        q_batch = [q] * len(d_texts[i:i+batch_size])
        d_batch = d_texts[i:i+batch_size]

        enc = ce_tok(
            q_batch,
            d_batch,
            padding="max_length",            # ensures uniform shape in a batch
            truncation="only_second",        # keep query, truncate passage
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        logits = ce_model(**enc).logits.squeeze(-1)
        scores.extend(logits.detach().float().cpu().tolist())

    out = df_candidates.copy()
    out["ce_score"] = scores
    return out.sort_values("ce_score", ascending=False).head(top_n).reset_index(drop=True)

# Build context block
def build_context_block(reranked_df: pd.DataFrame, max_chars: int = 2200) -> str:
    """Bullet snippets with source/URL; clipped to max_chars."""
    ctx = []
    total = 0
    for _, r in reranked_df.iterrows():
        snip = str(r["answer"]).strip()
        src  = (r.get("source","") or "Unknown").strip()
        url  = (r.get("url","") or "").strip()
        block = f"- {snip}\n  (Source: {src} {url})"
        if total + len(block) > max_chars:
            break
        ctx.append(block); total += len(block)
    return "\n".join(ctx)



OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacity of 14.56 GiB of which 16.81 MiB is free. Process 28705 has 3.63 GiB memory in use. Process 46195 has 5.53 GiB memory in use. Process 49664 has 1.50 GiB memory in use. Including non-PyTorch memory, this process has 3.88 GiB memory in use. Of the allocated memory 3.72 GiB is allocated by PyTorch, and 46.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# ########### ORIGINAL ############

# # --------------------------------------------------------
# # 🔹 Baseline 1 — Finetuned BioGPT (no RAG)
# # --------------------------------------------------------
# @torch.inference_mode()
# def generate_no_rag(user_q: str, tok, gen_model, max_new_tokens: int = 180) -> dict:
#     """
#     Baseline 1: Finetuned BioGPT (no retrieval).
#     The model relies only on what it learned during fine-tuning.
#     """
#     system = (
#         "You are a careful medical information assistant for the general public.\n"
#         "- Use short sentences and plain language.\n"
#         "- Avoid diagnosis; give general guidance and next steps.\n"
#         "- If you don't know, say you don't know.\n"
#     )
#     prompt = f"{system}\nQuestion: {user_q.strip()}\nAnswer:"
#     inputs = tok(prompt, return_tensors="pt").to(DEVICE)

#     out_ids = gen_model.generate(
#         **inputs,
#         do_sample=False,
#         max_new_tokens=max_new_tokens,
#         pad_token_id=tok.pad_token_id,
#         eos_token_id=tok.eos_token_id,
#         use_cache=True
#     )

#     text = tok.decode(out_ids[0], skip_special_tokens=True)
#     ans = text.split("Answer:", 1)
#     ans = (ans[1] if len(ans) > 1 else text).strip()
#     return {"answer": ans, "evidence": None}


# # --------------------------------------------------------
# # 🔹 Baseline 2 & Full Model — BioGPT + RAG
# # --------------------------------------------------------
# @torch.inference_mode()
# def generate_with_rag(
#     user_q: str,
#     tok, gen_model,
#     k_bm25=200, k_dense=200, k_final=50, alpha=0.5,
#     top_n=6,
#     max_new_tokens=220,
#     do_sample=False, top_p=1.0, repetition_penalty=1.02,
#     no_repeat_ngram_size=None
# ) -> dict:
#     cand = hybrid_search(user_q, k_bm25=k_bm25, k_dense=k_dense,
#                          k_final=k_final, alpha=alpha)
#     rer = rerank_cross_encoder(
#         user_q, cand,
#         top_n=top_n, batch_size=16, max_length=512, max_passage_tokens=400
#     )
#     prompt = build_rag_prompt(user_q, rer, tok=tok, ctx_token_budget=750)

#     inputs = tok(prompt, return_tensors="pt").to(DEVICE)
#     gen_kwargs = dict(
#         do_sample=do_sample,
#         top_p=top_p,
#         max_new_tokens=max_new_tokens,
#         repetition_penalty=repetition_penalty,
#         pad_token_id=tok.pad_token_id,
#         eos_token_id=tok.eos_token_id,
#         use_cache=True
#     )
#     if no_repeat_ngram_size is not None:
#         gen_kwargs["no_repeat_ngram_size"] = no_repeat_ngram_size

#     out_ids = gen_model.generate(**inputs, **gen_kwargs)
#     text = tok.decode(out_ids[0], skip_special_tokens=True)
#     ans = text.split("Answer:", 1)
#     ans = (ans[1] if len(ans) > 1 else text).strip()

#     cols = [c for c in ["answer","source","url","ce_score"] if c in rer.columns]
#     used = rer[cols].copy() if cols else pd.DataFrame(columns=["answer","source","url","ce_score"])

#     return {"answer": ans, "evidence": used}


In [ ]:
# =============================
# Finalized Generation Pipelines
# =============================


# Detect device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# ====== No RAG (Baseline 1) ======
@torch.inference_mode()
def generate_no_rag(user_q: str, tok, gen_model,
                    max_new_tokens: int = 150,
                    repetition_penalty: float = 1.1,
                    no_repeat_ngram_size: int = 3) -> dict:
    """
    Baseline 1: Finetuned BioGPT (no retrieval).
    The model relies only on what it learned during fine-tuning.
    """
    system = (
        "You are a careful medical information assistant for the general public.\n"
        "- Use short sentences and plain language.\n"
        "- Avoid diagnosis; give general guidance and next steps.\n"
        "- If you don't know, say you don't know.\n"
    )
    prompt = f"{system}\nQuestion: {user_q.strip()}\nAnswer:"

    # Tokenize safely
    inputs = tok(prompt, return_tensors="pt",
                 truncation=True, max_length=512).to(DEVICE)

    out_ids = gen_model.generate(
        **inputs,
        do_sample=False,  # greedy decoding
        max_new_tokens=max_new_tokens,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
        use_cache=True
    )

    text = tok.decode(out_ids[0], skip_special_tokens=True)
    ans = text.split("Answer:", 1)
    ans = (ans[1] if len(ans) > 1 else text).strip()
    return {"answer": ans, "evidence": None}


# ====== RAG (Baseline 2 + Full Model) ======
SYSTEM_RAG_FINE = (
    "You are a careful medical information assistant for the general public.\n"
    "- Answer ONLY using the Context bullets; if missing, say you don't know.\n"
    "- Use short sentences and plain language. Avoid diagnosis; give general guidance and next steps.\n"
    "- Add bracketed numeric citations [1], [2] that refer to the bullets you used.\n"
)

SYSTEM_RAG_BASE = (
    "Answer the following medical question briefly using only the context below. "
    "If the context does not contain the answer, reply: 'I don't know.'\n"
)


def build_rag_prompt(user_q: str,
                     reranked_df: pd.DataFrame,
                     tok,
                     ctx_token_budget: int = 700,
                     system_text: str = SYSTEM_RAG_FINE) -> str:
    """
    Build a concise RAG prompt with a safe token budget.
    """
    bullets, used = [], 0
    for i, (_, r) in enumerate(reranked_df.iterrows(), start=1):
        block = f"[{i}] {str(r['answer']).strip()}\n" \
                f"(Source: {(r.get('source') or 'Unknown').strip()} {(r.get('url') or '').strip()})"
        ids = tok.encode(block, add_special_tokens=False)
        if used + len(ids) > ctx_token_budget:
            break
        bullets.append(block)
        used += len(ids)

    ctx = "\n".join(bullets)
    return (
        f"{system_text}\n"
        f"Context:\n{ctx}\n\n"
        f"Question: {user_q.strip()}\n"
        f"Answer:"
    )


@torch.inference_mode()
def generate_with_rag(
    user_q: str,
    tok,
    gen_model,
    use_base_prompt: bool = False,  # True if using BioGPT-base (zero-shot)
    k_bm25=200, k_dense=200, k_final=50, alpha=0.5,
    top_n=6,
    max_new_tokens=150,
    do_sample=False,
    top_p=1.0,
    repetition_penalty=1.1,
    no_repeat_ngram_size=3
) -> dict:
    """
    Full RAG pipeline for Baseline 2 or Full Model.
    Returns: {"answer": str, "evidence": pd.DataFrame[answer, source, url, ce_score]}
    """
    # --- 1. Hybrid retrieval ---
    cand = hybrid_search(user_q, k_bm25=k_bm25,
                         k_dense=k_dense, k_final=k_final, alpha=alpha)

    # --- 2. Cross-encoder re-ranking ---
    rer = rerank_cross_encoder(
        user_q, cand, top_n=top_n,
        batch_size=16, max_length=512, max_passage_tokens=400
    )

    # --- 3. Build RAG prompt ---
    system_text = SYSTEM_RAG_BASE if use_base_prompt else SYSTEM_RAG_FINE
    prompt = build_rag_prompt(user_q, rer, tok=tok,
                              ctx_token_budget=700, system_text=system_text)

    # --- 4. Generate ---
    inputs = tok(prompt, return_tensors="pt",
                 truncation=True, max_length=512).to(DEVICE)

    gen_kwargs = dict(
        do_sample=do_sample,
        top_p=top_p,
        max_new_tokens=max_new_tokens,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id,
        use_cache=True
    )

    out_ids = gen_model.generate(**inputs, **gen_kwargs)
    text = tok.decode(out_ids[0], skip_special_tokens=True)

    ans = text.split("Answer:", 1)
    ans = (ans[1] if len(ans) > 1 else text).strip()

    # --- 5. Extract evidence ---
    cols = [c for c in ["answer", "source", "url", "ce_score"] if c in rer.columns]
    used = rer[cols].copy() if cols else pd.DataFrame(
        columns=["answer", "source", "url", "ce_score"]
    )

    return {"answer": ans, "evidence": used}


In [ ]:
# Evaluation Metrics
nltk.download('stopwords')
from nltk.corpus import stopwords

rouge = evaluate.load("rouge")          # ROUGE-1/2/L
stop_words = set(stopwords.words("english"))
smooth = SmoothingFunction().method3    # BLEU smoothing for short answers

def normalize_text(s: str) -> str:
    s = s.lower()
    s = s.translate(str.maketrans("", "", string.punctuation))
    s = " ".join(s.split())
    return s

def exact_match(pred: str, gold: str) -> int:
    return int(normalize_text(pred) == normalize_text(gold))

def token_f1(pred: str, gold: str) -> float:
    pt = normalize_text(pred).split()
    gt = normalize_text(gold).split()
    common = set(pt) & set(gt)
    num_same = sum(min(pt.count(w), gt.count(w)) for w in common)
    if not pt or not gt:
        return float(pt == gt)
    if num_same == 0:
        return 0.0
    precision = num_same / len(pt)
    recall    = num_same / len(gt)
    return 2 * precision * recall / (precision + recall)

def bleu_score(pred: str, gold: str) -> float:
    # sentence_bleu expects list of references (each is list of tokens)
    ref = normalize_text(gold).split()
    hyp = normalize_text(pred).split()
    if not hyp:
        return 0.0
    return sentence_bleu([ref], hyp, smoothing_function=smooth)

def rouge_l(preds, refs) -> float:
    # returns ROUGE-L fmeasure
    out = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
    return float(out["rougeL"])


def content_tokens(s: str):
    toks = normalize_text(s).split()
    return [t for t in toks if (t not in stop_words) and (t.isalpha())]

def support_ratio(pred: str, contexts: list[str]) -> float:
    pred_ct = content_tokens(pred)
    if not pred_ct:
        return 1.0  # empty prediction = not hallucinated
    ctx = " ".join(contexts)
    ctx_set = set(content_tokens(ctx))
    supported = sum(1 for t in pred_ct if t in ctx_set)
    return supported / max(1, len(pred_ct))

def hallucinated(pred: str, contexts: list[str], thresh: float = 0.6) -> int:
    return int(support_ratio(pred, contexts) < thresh)

bertscore = evaluate.load("bertscore")

def bert_score(preds, refs, lang="en", model_type="bert-base-uncased"):
    """
    preds: list[str] of candidate answers
    refs:  list[str] of reference answers
    returns: dict with keys 'precision', 'recall', 'f1'
    """
    result = bertscore.compute(
        predictions=preds,
        references=refs,
        lang=lang,
        model_type=model_type
    )
    return result

In [ ]:
# --- Unified evaluator for Baselines & Full Model (fixed) ---


def evaluate_model(
    eval_df: pd.DataFrame,
    generation_fn,                          # <-- function comes 2nd now
    name: str,
    save_path: str = None
) -> Tuple[Dict, pd.DataFrame]:
    """
    Evaluate a generator function over a validation dataframe.
    generation_fn: callable(q) -> {"answer": str, "evidence": pd.DataFrame or None}
    Returns: (metrics_dict, detailed_df)
    """
    preds, refs = [], []
    em_list, f1_list, bleu_list = [], [], []
    supp_list, hall_list = [], []
    rows_out = []

    print(f"\n=== Evaluating {name} on {len(eval_df)} samples ===")

    for _, row in tqdm(eval_df.iterrows(), total=len(eval_df), desc=f"Eval {name}"):
        q = str(row["question"])
        gold = str(row["answer"])

        # --- Generate ---
        try:
            out = generation_fn(q)
            if not isinstance(out, dict):
                raise TypeError(f"generator_fn returned {type(out)}, expected dict")
            pred = str(out.get("answer", "") or "").strip()
            ev_df = out.get("evidence", None)
        except Exception as e:
            print(f"[WARN] Generation failed for: {q[:60]}... ({e})")
            pred, ev_df = "", None

        # --- Context from retrieved evidence (if any) ---
        ctxs = []
        if ev_df is not None:
            try:
                if hasattr(ev_df, "empty") and not ev_df.empty and "answer" in ev_df.columns:
                    ctxs = ev_df["answer"].astype(str).tolist()
            except Exception:
                ctxs = []

        # --- Metrics ---
        preds.append(pred)
        refs.append(gold)
        try:
            em_list.append(exact_match(pred, gold))
        except Exception:
            em_list.append(0.0)
        try:
            f1_list.append(token_f1(pred, gold))
        except Exception:
            f1_list.append(0.0)
        try:
            bleu_list.append(bleu_score(pred, gold))
        except Exception:
            bleu_list.append(0.0)
        try:
            supp_list.append(support_ratio(pred, ctxs))
        except Exception:
            supp_list.append(0.0)
        try:
            hall_list.append(hallucinated(pred, ctxs, thresh=0.6))
        except Exception:
            hall_list.append(1.0)  # pessimistic default if check fails

        # --- Record per sample (safe evidence extraction) ---
        def _safe_ev(i):
            try:
                return ev_df.iloc[i]["answer"] if ev_df is not None and len(ev_df) > i and "answer" in ev_df.columns else ""
            except Exception:
                return ""

        rows_out.append({
            "question": q,
            "gold_answer": gold,
            "prediction": pred,
            "EM": em_list[-1],
            "F1": f1_list[-1],
            "BLEU": bleu_list[-1],
            "SupportRatio": supp_list[-1],
            "Hallucinated": hall_list[-1],
            "evidence_1": _safe_ev(0),
            "evidence_2": _safe_ev(1),
            "evidence_3": _safe_ev(2),
        })

    # --- Aggregate Metrics ---
    try:
        rouge_l_f = rouge_l(preds, refs)
    except Exception:
        rouge_l_f = 0.0

    # BERTScore (skip if all preds empty to avoid noisy warnings)
    try:
        any_pred_text = any(bool(p.strip()) for p in preds)
        if any_pred_text:
            bert_res = bert_score(preds, refs)
            bert_p = float(np.mean(bert_res.get("precision", [0.0])))
            bert_r = float(np.mean(bert_res.get("recall", [0.0])))
            bert_f = float(np.mean(bert_res.get("f1", [0.0])))
        else:
            bert_p = bert_r = bert_f = 0.0
    except Exception:
        bert_p = bert_r = bert_f = 0.0

    results = {
        "N": len(eval_df),
        "ExactMatch": float(np.mean(em_list)) if em_list else 0.0,
        "TokenF1": float(np.mean(f1_list)) if f1_list else 0.0,
        "BLEU": float(np.mean(bleu_list)) if bleu_list else 0.0,
        "ROUGE-L": float(rouge_l_f),
        "SupportRatio(avg)": float(np.mean(supp_list)) if supp_list else 0.0,
        "HallucinationRate(<0.6 support)": float(np.mean(hall_list)) if hall_list else 0.0,
        "BERTScore_P": bert_p,
        "BERTScore_R": bert_r,
        "BERTScore_F1": bert_f,
    }

    # --- Save optional CSV ---
    details = pd.DataFrame(rows_out)
    if save_path:
        try:
            details.to_csv(save_path, index=False)
            print(f"Saved per-sample details to {save_path}")
        except Exception as e:
            print(f"[WARN] Failed to save CSV to {save_path}: {e}")

    print(f"\n=== {name} Summary ===")
    for k, v in results.items():
        print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

    return results, details


In [ ]:
# MODELS

# --- Baseline 1 (fine-tuned BioGPT only) ---
gen_fn_baseline1 = lambda q: generate_no_rag(q, tok_full, gen_full)

# --- Baseline 2 (RAG + zero-shot BioGPT) ---
gen_fn_baseline2 = lambda q: generate_with_rag(q, tok_base, gen_base)

# --- Full Model (RAG + fine-tuned BioGPT) ---
gen_fn_full = lambda q: generate_with_rag(q, tok_full, gen_full)


In [ ]:
baseline1_results = evaluate_model(val_df, gen_fn_baseline1, "Baseline 1 – Finetuned BioGPT", "baseline1_val.csv")


In [ ]:
baseline2_results = evaluate_model(val_df, gen_fn_baseline2, "Baseline 2 – RAG + Base BioGPT", "baseline2_val.csv")


In [ ]:
fullmodel_results  = evaluate_model(val_df, gen_fn_full, "Full Model – RAG + Finetuned BioGPT", "fullmodel_val.csv")


In [ ]:
baseline1_results, baseline1_details = baseline1_results
baseline2_results, baseline2_details = baseline2_results
fullmodel_results, fullmodel_details = fullmodel_results

summary_df = pd.DataFrame([
    {"Model": "Baseline 1 (Fine-tuned only)", **baseline1_results},
    {"Model": "Baseline 2 (RAG + Base BioGPT)", **baseline2_results},
    {"Model": "Full Model (RAG + Fine-tuned)", **fullmodel_results}
])

# Drop the “N” column safely if it exists
summary_df.drop(columns=["N"], inplace=True, errors="ignore")

# Save to CSV
summary_df.to_csv("model_comparison_summary.csv", index=False)

summary_df
